In [1]:
import numpy as np
import pandas as pd

In [2]:
import random
import pickle

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Data

In [4]:
ufo_df = pd.read_csv('data/ufo_df.csv', index_col=0)

In [5]:
ufo_df.head()

,datetime,geolocation,season,month,day,time_of_day,region,shape,duration,report_text
report_id,,,,,,,,,,
073/S73909,2010-01-01 00:00:00,"Park City, KY",Winter,January,Friday,Night,East South Central,Light,3.0,"Lights orbiting the moon,I am 10,and i was wit..."
073/S73915,2010-01-01 00:00:00,"La Mesa, CA",Winter,January,Friday,Night,Pacific,Light,600.0,Three red lights over southern California that...
078/S78231,2010-01-01 00:00:00,"Benton, AR",Winter,January,Friday,Night,West South Central,Circle,300.0,4 bright green circles high in the sky going i...
073/S73918,2010-01-01 00:00:00,"El Cajon, CA",Winter,January,Friday,Night,Pacific,Triangle,720.0,"3 Red objects hovering over El Cajon CA ,Exit..."
073/S73916,2010-01-01 00:00:00,"Lemon Grove, CA",Winter,January,Friday,Night,Pacific,Light,900.0,3 Red lights in line pattern above El Cajon/ E...


# Vectorize Report Text
Create sparse matrix of word counts for each document (report)

In [6]:
# text data
documents = ufo_df['report_text']

In [7]:
# # create instance of TFIDF vectorizer
# tf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=0.05, ngram_range=(1, 3))
# # transform documents into document-term matrix
# X = tf_vectorizer.fit_transform(documents)
# # save vectorizer
# with open('model_output/tfidf.pkl', 'wb') as outfile:
#     pickle.dump(tf_vectorizer, outfile)
# # save array
# with open('model_output/v_array.pkl', 'wb') as outfile:
#     pickle.dump(X, outfile)

In [8]:
# load vectorizer
with open('model_output/tfidf.pkl', 'rb') as infile:
    tf_vectorizer = pickle.load(infile)
# load vectorized array
with open('model_output/v_array.pkl', 'rb') as infile:
    X = pickle.load(infile)

In [9]:
# create dataframe of document-term matrix
tf_df = pd.DataFrame(X.toarray(), columns=[tf_vectorizer.get_feature_names()])
tf_df.head()

,10,12,15,20,30,45,able,air,aircraft,airplane,...,white,white light,white lights,wife,window,witness,witness elects,witness elects remain,witnessed,yellow
0,0.137760,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.230315,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0.143400,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.189832,0.000000,0.0,0.0,0.0,0.175565,0.0
2,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.117303,0.0,0.0,0.0,0.000000,0.0
3,0.000000,0.24191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
4,0.113088,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.276908,0.0


# Topic Modeling
Generate topics

In [10]:
# function to print top words of topic model
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}: ".format(index)
        message += "; ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)

In [11]:
# # create instance of model, input number of topics to output
# nmf = NMF(n_components=10, random_state=0)
# # run model
# nmf.fit(tf_df)
# # save model
# with open('model_output/nmf.pkl', 'wb') as outfile:
#     pickle.dump(nmf, outfile)

In [12]:
# load model
with open('model_output/nmf.pkl', 'rb') as infile:
    nmf = pickle.load(infile)

In [13]:
# create array of topic weights per document
topic_array = nmf.transform(tf_df)

In [14]:
# display top words for each topic
print_top_words(nmf, tf_vectorizer.get_feature_names(), 20)


Topic #0: object; shaped; appeared; sky; shape; observed; white; large; approximately; did; noticed; stationary; hovering; circular; size; round; bright; view; color; minutes

Topic #1: information; provides; contact information; anonymous; pd; nuforc note; elects; nuforc; elects remain; contact; remain; anonymous provides; nuforc note witness; note witness; totally anonymous; remain totally; remain totally anonymous; elects remain totally; note; information pd

Topic #2: lights; formation; triangle; white lights; orange lights; flashing; white; blinking; shape; red lights; line; triangular; sky; saw; appeared; driving; moving; hovering; aircraft; noticed

Topic #3: light; bright; white; bright light; white light; sky; bright white; star; flash; disappeared; blue; appeared; seconds; brighter; moved; minutes; stationary; noticed; night; seen

Topic #4: orange; bright orange; orange lights; glowing; sky; disappeared; ball; fireball; bright; orb; slowly; minutes; appeared; saw; color; fa

## Greatest Topic

In [15]:
# topic label dictionary
topic_dict = {
    0: 'Object Description',
    1: 'NUFORC Notes',
    2: 'Lights 1',
    3: 'Lights 2',
    4: 'Fiery Lights',
    5: 'Observation Terms',
    6: 'Craft Description',
    7: 'Observation Description',
    8: 'Colors',
    9: 'Direction'
}

In [16]:
# create topic dataframe
topic_df = pd.DataFrame(topic_array, columns=topic_dict.values(), index=ufo_df.index)
# convert topic weights to percentile per document
topic_df = round(topic_df.div(topic_df.sum(axis=1), axis=0)*100, 0)
topic_df.head()

,Object Description,NUFORC Notes,Lights 1,Lights 2,Fiery Lights,Observation Terms,Craft Description,Observation Description,Colors,Direction
report_id,,,,,,,,,,
073/S73909,3.0,0.0,26.0,16.0,6.0,36.0,0.0,0.0,7.0,7.0
073/S73915,0.0,0.0,43.0,0.0,3.0,12.0,1.0,2.0,26.0,13.0
078/S78231,0.0,0.0,0.0,49.0,0.0,20.0,0.0,0.0,31.0,0.0
073/S73918,0.0,0.0,1.0,0.0,0.0,5.0,0.0,70.0,24.0,0.0
073/S73916,0.0,0.0,23.0,15.0,1.0,17.0,0.0,4.0,24.0,17.0


In [17]:
# assign dominant topic to report observation
ufo_tops = topic_df.idxmax(axis=1)

In [18]:
# number of missing topics
ufo_tops.isna().sum()

5

In [19]:
# replace empty with most common
ufo_tops.fillna(ufo_tops.mode().values[0], inplace=True)

In [20]:
# export topic series
# ufo_tops.to_csv('data/ufo_tops.csv', header=False)